## Building Technical Strategies with VectorBT

In [ ]:
# !pip install vectorbt

In [1]:
import pandas as pd
import vectorbt as vbt
from IPython.display import Markdown, display

Define the start and end dates for data download

In [2]:
start = "2016-01-01 UTC"
end = "2020-01-01 UTC"

Download historical closing prices for specified symbols from Yahoo Finance

In [3]:
prices = vbt.YFData.download(
    ["META", "AAPL", "AMZN", "NFLX", "GOOG"], start=start, end=end
).get("Close")

In [4]:
prices

symbol,META,AAPL,AMZN,NFLX,GOOG
Date,,,,,
2016-01-04 05:00:00+00:00,101.831017,23.834383,31.849501,109.959999,36.959198
2016-01-05 05:00:00+00:00,102.339073,23.237103,31.689501,107.660004,36.996067
2016-01-06 05:00:00+00:00,102.578156,22.782360,31.632500,117.680000,37.047882
2016-01-07 05:00:00+00:00,97.547379,21.820843,30.396999,114.559998,36.189465
2016-01-08 05:00:00+00:00,96.959625,21.936226,30.352501,111.389999,35.595604
...,...,...,...,...,...
2019-12-24 05:00:00+00:00,204.339447,68.823021,89.460503,333.200012,66.937485
2019-12-26 05:00:00+00:00,206.999268,70.188492,93.438499,332.630005,67.776466
2019-12-27 05:00:00+00:00,207.308105,70.161858,93.489998,329.089996,67.352501


Calculate fast and slow moving averages

In [5]:
fast_ma = vbt.MA.run(prices, 10, short_name="fast")
slow_ma = vbt.MA.run(prices, 30, short_name="slow")

Generate entry signals when the fast MA crosses above the slow MA

In [6]:
entries = fast_ma.ma_crossed_above(slow_ma)
display(entries)

fast_window                   10                            
slow_window                   30                            
symbol                      META   AAPL   AMZN   NFLX   GOOG
Date                                                        
2016-01-04 05:00:00+00:00  False  False  False  False  False
2016-01-05 05:00:00+00:00  False  False  False  False  False
2016-01-06 05:00:00+00:00  False  False  False  False  False
2016-01-07 05:00:00+00:00  False  False  False  False  False
2016-01-08 05:00:00+00:00  False  False  False  False  False
...                          ...    ...    ...    ...    ...
2019-12-24 05:00:00+00:00  False  False  False  False  False
2019-12-26 05:00:00+00:00  False  False  False  False  False
2019-12-27 05:00:00+00:00  False  False  False  False  False
2019-12-30 05:00:00+00:00  False  False  False  False  False
2019-12-31 05:00:00+00:00  False  False  False  False  False

[1006 rows x 5 columns]

Generate exit signals when the fast MA crosses below the slow MA

In [7]:
exits = fast_ma.ma_crossed_below(slow_ma)
display(exits)

fast_window                   10                            
slow_window                   30                            
symbol                      META   AAPL   AMZN   NFLX   GOOG
Date                                                        
2016-01-04 05:00:00+00:00  False  False  False  False  False
2016-01-05 05:00:00+00:00  False  False  False  False  False
2016-01-06 05:00:00+00:00  False  False  False  False  False
2016-01-07 05:00:00+00:00  False  False  False  False  False
2016-01-08 05:00:00+00:00  False  False  False  False  False
...                          ...    ...    ...    ...    ...
2019-12-24 05:00:00+00:00  False  False  False  False  False
2019-12-26 05:00:00+00:00  False  False  False  False  False
2019-12-27 05:00:00+00:00  False  False  False  False  False
2019-12-30 05:00:00+00:00  False  False  False  False  False
2019-12-31 05:00:00+00:00  False  False  False  False  False

[1006 rows x 5 columns]

Create a portfolio using the generated entry and exit signals

In [8]:
# Criando um portfólio a partir dos sinais de entrada e saída
# close: preços de fechamento das ações
# entries: sinais de entrada (quando cruzamento da média rápida acima da lenta)
# exits: sinais de saída (quando cruzamento da média rápida abaixo da lenta)
# freq: frequência dos dados (diária)
pf = vbt.Portfolio.from_signals(
    close=prices,
    entries=entries,
    exits=exits,
    freq="1d"
)

Display portfolio statistics

In [9]:
display(pf.stats())

/tmp/ipykernel_7720/4100483198.py:1: UserWarning: Object has multiple columns. Aggregating using <function mean at 0x7fd050f567a0>. Pass column to select a single column/group.
  display(pf.stats())


Start                          2016-01-04 05:00:00+00:00
End                            2019-12-31 05:00:00+00:00
Period                                1006 days 00:00:00
Start Value                                        100.0
End Value                                     176.376637
Total Return [%]                               76.376637
Benchmark Return [%]                          152.731359
Max Gross Exposure [%]                             100.0
Total Fees Paid                                      0.0
Max Drawdown [%]                               25.869062
Max Drawdown Duration                  347 days 00:00:00
Total Trades                                        17.4
Total Closed Trades                                 16.4
Total Open Trades                                    1.0
Open Trade PnL                                 23.097846
Win Rate [%]                                    49.79085
Best Trade [%]                                 27.628849
Worst Trade [%]                

Plot the total return of the portfolio for each symbol

In [10]:
(pf.total_return().groupby("symbol").mean().vbt.barplot())

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': '0da4aeeb-3131-4fa3-8f8e-8db56a371e82',
              'x': array(['AAPL', 'AMZN', 'GOOG', 'META', 'NFLX'], dtype=object),
              'y': array([1.73482874, 1.28553102, 0.50135246, 0.16619091, 0.13092872])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

Create a portfolio assuming buy-and-hold strategy and plot the total return

In [21]:
pf2 =vbt.Portfolio.from_holding(prices, freq="1d")

In [ ]:
(
    vbt.Portfolio.from_holding(prices, freq="1d")
    .total_return()
    .groupby("symbol")
    .mean()
    .vbt.barplot()
)

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': '30880829-a672-4e97-88ee-c4939ad83daf',
              'x': array(['AAPL', 'AMZN', 'GOOG', 'META', 'NFLX'], dtype=object),
              'y': array([1.98283278, 1.90089315, 0.80230253, 1.00792391, 1.94261559])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

## Running split tests

Split the prices data into 4 equal parts

In [30]:
mult_prices, _ = prices.vbt.range_split(n=4)

In [31]:
mult_prices

split_idx           0                                               \
symbol           META       AAPL       AMZN        NFLX       GOOG   
0          101.831017  23.834383  31.849501  109.959999  36.959198   
1          102.339073  23.237103  31.689501  107.660004  36.996067   
2          102.578156  22.782360  31.632500  117.680000  37.047882   
3           97.547379  21.820843  30.396999  114.559998  36.189465   
4           96.959625  21.936226  30.352501  111.389999  35.595604   
..                ...        ...        ...         ...        ...   
246        116.953247  26.894806  38.317001  125.580002  39.421356   
247        116.823738  26.948000  38.029499  125.589996  39.354095   
248        117.560928  27.119141  38.570000  128.350006  39.435806   
249        116.475075  27.003504  38.606499  125.889999  39.111965   
250        115.907242  26.996571  38.257500  125.330002  38.999374   

split_idx           1                                               \
symbol           META       AAPL       AMZN        NFLX       GOOG   
0          116.415298  26.862427  37.683498  127.489998  39.166271   
1          118.238335  26.832352  37.859001  129.410004  39.204140   
2          120.210800  26.968811  39.022499  131.809998  39.558861   
3          122.940384  27.269466  39.799500  131.070007  40.163185   
4          124.424713  27.519241  39.846001  130.949997  40.188099   
..                ...        ...        ...         ...        ...   
246        176.525696  41.126072  58.417999  189.940002  52.816227   
247        175.320297  40.082703  58.838001  187.759995  52.647835   
248        176.944061  40.089752  59.112999  186.240005  52.280651   
249        177.242950  40.202549  59.305000  192.710007  52.219372   
250        175.788513  39.767807  58.473499  191.960007  52.132683   

split_idx           2                                               \
symbol           META       AAPL       AMZN        NFLX       GOOG   
0          180.729614  40.479836  59.450500  201.070007  53.059353   
1          183.967255  40.472797  60.209999  205.050003  53.930225   
2          183.628555  40.660786  60.479500  205.630005  54.125523   
3          186.138962  41.123718  61.457001  209.990005  54.914188   
4          187.563507  40.970974  62.343498  212.050003  55.148846   
..                ...        ...        ...         ...        ...   
246        123.587891  35.021992  67.197998  233.880005  48.636250   
247        133.669388  37.488277  73.544998  253.669998  51.786926   
248        134.008102  37.244991  73.082001  255.570007  52.007133   
249        132.693130  37.264080  73.901001  256.079987  51.668350   
250        130.591141  37.624249  75.098503  267.660004  51.595112   

split_idx           3                                               
symbol           META       AAPL       AMZN        NFLX       GOOG  
0          131.238678  33.915257  75.014000  271.200012  50.621113  
1          137.425034  35.363064  78.769501  297.570007  53.343830  
2          137.524673  35.284367  81.475502  315.339996  53.228245  
3          141.987625  35.956982  82.829002  320.269989  53.621330  
4          143.681137  36.567604  82.971001  319.959991  53.540623  
..                ...        ...        ...         ...        ...  
246        204.339447  68.823021  89.460503  333.200012  66.937485  
247        206.999268  70.188492  93.438499  332.630005  67.776466  
248        207.308105  70.161858  93.489998  329.089996  67.352501  
249        203.632141  70.578278  92.344498  323.309998  66.567810  
250        204.468933  71.093979  92.391998  323.570007  66.611656  

[251 rows x 20 columns]

Calculate fast and slow moving averages for each split

In [32]:
fast_ma = vbt.MA.run(mult_prices, [10, 20], short_name="fast")
slow_ma = vbt.MA.run(mult_prices, [30, 30], short_name="slow")

Generate entry and exit signals for each split

In [33]:
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

Create a portfolio using the generated entry and exit signals for each split

In [34]:
pf = vbt.Portfolio.from_signals(mult_prices, entries, exits, freq="1D")

In [39]:
pf.total_return().groupby(["split_idx", "symbol"]).mean()

split_idx  symbol
0          AAPL      0.050632
           AMZN      0.301214
           GOOG     -0.082265
           META     -0.041474
           NFLX     -0.058673
1          AAPL      0.113215
           AMZN      0.109763
           GOOG      0.178351
           META      0.080160
           NFLX     -0.036279
2          AAPL      0.105406
           AMZN      0.262656
           GOOG      0.011484
           META     -0.136146
           NFLX      0.043942
3          AAPL      0.441026
           AMZN      0.010399
           GOOG      0.131536
           META      0.102773
           NFLX     -0.072186
Name: total_return, dtype: float64

In [40]:
pf.total_return().groupby(["split_idx", "symbol"]).mean().unstack(level=-1)

symbol,AAPL,AMZN,GOOG,META,NFLX
split_idx,,,,,
0,0.050632,0.301214,-0.082265,-0.041474,-0.058673
1,0.113215,0.109763,0.178351,0.080160,-0.036279
2,0.105406,0.262656,0.011484,-0.136146,0.043942
3,0.441026,0.010399,0.131536,0.102773,-0.072186


Plot the total return of the portfolio for each split and symbol

In [37]:
(
    pf.total_return()
    .groupby(["split_idx", "symbol"])
    .mean()
    .unstack(level=-1)
    .vbt.barplot()
)

FigureWidget({
    'data': [{'name': 'AAPL',
              'showlegend': True,
              'type': 'bar',
              'uid': '0f1aa45a-0a8d-4e65-b4c3-b6c08c6ae9c6',
              'x': array([0, 1, 2, 3]),
              'y': array([0.05063199, 0.11321507, 0.10540587, 0.44102596])},
             {'name': 'AMZN',
              'showlegend': True,
              'type': 'bar',
              'uid': '62b1d45d-7a67-44f4-81c4-e06461d4e837',
              'x': array([0, 1, 2, 3]),
              'y': array([0.30121359, 0.10976334, 0.26265571, 0.01039871])},
             {'name': 'GOOG',
              'showlegend': True,
              'type': 'bar',
              'uid': '34c963a3-a649-4d9c-9ee4-ef01fcf9d11e',
              'x': array([0, 1, 2, 3]),
              'y': array([-0.08226518,  0.17835128,  0.0114837 ,  0.13153604])},
             {'name': 'META',
              'showlegend': True,
              'type': 'bar',
              'uid': 'e01a7dcf-0561-461a-9f84-68a5037e1b4c',
              

Display order statistics

In [44]:
# Statistics of the orders
display(pf.orders.stats(group_by=True))

Start                                0
End                                250
Period               251 days 00:00:00
Total Records                      291
Total Buy Orders                   157
Total Sell Orders                  134
Min Size                      0.253075
Max Size                      4.711689
Avg Size                      1.635554
Avg Buy Size                  1.644335
Avg Sell Size                 1.625266
Avg Buy Price               103.305937
Avg Sell Price              104.427337
Total Fees                         0.0
Min Fees                           0.0
Max Fees                           0.0
Avg Fees                           0.0
Avg Buy Fees                       0.0
Avg Sell Fees                      0.0
Name: group, dtype: object

Display the Sharpe ratio of the portfolio

In [45]:
# Statistics of the portfolio
display(pf.stats())

/tmp/ipykernel_7720/4100483198.py:1: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x7fd050f567a0>. Pass column to select a single column/group.



Start                                                  0
End                                                  250
Period                                 251 days 00:00:00
Start Value                                        100.0
End Value                                     107.577668
Total Return [%]                                7.577668
Benchmark Return [%]                           28.298316
Max Gross Exposure [%]                             100.0
Total Fees Paid                                      0.0
Max Drawdown [%]                               15.474947
Max Drawdown Duration                  109 days 07:12:00
Total Trades                                       3.925
Total Closed Trades                                 3.35
Total Open Trades                                  0.575
Open Trade PnL                                  4.469044
Win Rate [%]                                        51.5
Best Trade [%]                                  9.579418
Worst Trade [%]                

In [42]:
display(pf.sharpe_ratio())

fast_window  slow_window  split_idx  symbol
10           30           0          META     -0.472596
                                     AAPL      1.125229
                                     AMZN      1.633329
                                     NFLX     -0.470585
                                     GOOG     -0.051109
                          1          META      0.449942
                                     AAPL      1.043131
                                     AMZN      0.611364
                                     NFLX     -0.359590
                                     GOOG      1.754601
                          2          META     -0.572692
                                     AAPL      0.049774
                                     AMZN      1.255488
                                     NFLX     -0.171628
                                     GOOG      0.088256
                          3          META      0.556394
                                     AAPL      2.921358
    

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.